In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
from utils import chart_price, plot, v_backtester, c_backtester, perf, perf_var, breakout_strategy, bootstrap, m_proc
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
import matplotlib.pyplot as plt
from typing import NamedTuple
from collections import namedtuple
%matplotlib inline

In [2]:
from datastore_pytables import Store

In [3]:
store = Store()
store.keys()

['/cont/min/CL_20191120_NYMEX_USD',
 '/cont/min/ES_20191220_GLOBEX_USD',
 '/cont/min/GC_20191227_NYMEX_USD',
 '/cont/min/GE_20191216_GLOBEX_USD',
 '/cont/min/NKD_20191212_GLOBEX_USD',
 '/cont/min/NQ_20191220_GLOBEX_USD',
 '/cont/min/YM_20191220_ECBOT_USD',
 '/cont/min/ZB_20191219_ECBOT_USD',
 '/cont/min/ZF_20191231_ECBOT_USD',
 '/cont/min/ZN_20191219_ECBOT_USD']

In [4]:
contract = store.read('/cont/min/CL_20191120_NYMEX_USD').sort_index()

In [5]:
table = bootstrap(contract,
          start = '20180101', 
          end = '20181231',
          paths=100)

In [6]:
func = partial(breakout_strategy, 
                  time_int = 45,
                  periods = [5, 10, 20, 40, 80, 160],
                  ema_fast = 30,
                  ema_slow = 120,
                  atr_periods = 180,
                  sl_atr = 1,
                  take_profit=0
                  )

In [7]:
results = m_proc(table, func)

In [8]:
res = pd.DataFrame([perf(i, output=False).stats for i in results])

In [9]:
res[res['Annual return'] > 0]['Annual return'].count()

95

In [10]:
res.sort_values('Annual return')

,Annual return,Cumulative returns,Annual volatility,Sharpe ratio,Calmar ratio,Stability,Max drawdown,Omega ratio,Sortino ratio,Skew,...,Avg gain/loss ratio,Position EV,Positions per day,Days per position,Actual avg. duration,Days,Positions,Trades,Monthly EV,Annual EV
92,-0.128196,-0.090874,0.157899,-0.791223,-0.981213,0.040933,-0.130650,0.845683,-1.364169,2.306477,...,1.783156,-0.022444,1.360000,0.735294,05:38:00,175,238,476,-0.641008,-7.692096
73,-0.058295,-0.040395,0.236223,-0.137692,-0.224248,0.129686,-0.259957,0.971948,-0.215290,0.854599,...,1.430799,-0.010507,1.289017,0.775785,05:59:00,173,223,446,-0.284423,-3.413073
61,-0.053962,-0.037154,0.183999,-0.209990,-0.426697,0.473670,-0.126464,0.957793,-0.311672,0.096678,...,1.469957,-0.010217,1.284884,0.778281,06:02:00,172,221,442,-0.275682,-3.308188
40,-0.037911,-0.026184,0.155538,-0.171844,-0.392853,0.054501,-0.096502,0.965713,-0.283626,1.466196,...,1.610047,-0.006802,1.265896,0.789954,06:15:00,173,219,437,-0.180810,-2.169725
17,-0.025465,-0.017552,0.175998,-0.059800,-0.159734,0.557599,-0.159421,0.988949,-0.098954,1.195741,...,1.956251,-0.004327,1.352601,0.739316,06:01:00,173,234,467,-0.122918,-1.475021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,0.650512,0.407775,0.125778,4.049690,29.771379,0.935525,-0.021850,2.320556,10.070853,1.785687,...,2.151096,0.108864,1.279070,0.781818,06:35:00,172,220,439,2.924144,35.089733
52,0.698645,0.438690,0.145530,3.715915,17.448863,0.874918,-0.040040,2.232957,9.453538,1.866997,...,2.193761,0.100728,1.300578,0.768889,05:55:00,173,225,450,2.751086,33.013027
7,0.750311,0.471856,0.165213,3.473143,13.772412,0.944334,-0.054479,2.244479,7.921641,1.636537,...,2.335902,0.125204,1.224138,0.816901,06:40:00,174,213,426,3.218610,38.623323
82,0.807326,0.497744,0.136958,4.394101,15.004662,0.965733,-0.053805,2.306646,9.437806,0.709290,...,2.245588,0.139570,1.209302,0.826923,07:22:00,172,208,416,3.544429,42.533143


In [11]:
res['Sharpe ratio'].median()

1.8521310629752508

In [12]:
res['Sharpe ratio'].mean()

1.8348124642948207

In [13]:
res[res['Annual return'] >=.2]['Annual return'].count()

70

In [14]:
res['Position EV'].median()

0.05473856372411143

In [15]:
res['Position EV'].mean()

0.05316687299861702

In [16]:
(res[res['Position EV'] < 2])['Annual return'].count()

100

In [17]:
(res[res['Position EV'] < 5])['Annual return'].count()

100

In [18]:
res['Monthly EV'].median()

1.5211752776048901

In [19]:
res['Monthly EV'].mean()

1.455786026167657